# AiiDA DB Query

Here I'm just collecting notes and simple examples of how to build queries in aiida. 

In [1]:
from aiida import load_dbenv
load_dbenv()

In [66]:
from aiida.orm import Calculation, Code, Computer, Data, Node
from aiida.orm import CalculationFactory, DataFactory
from aiida.backends.djsite.db import models
from aiida.orm.querybuilder import QueryBuilder
from datetime import datetime
from aiida.orm.utils import load_node

## StructureData

This subclass of Node contains structure data.

Before we build a query, let's see what attributes this subclass contains. I happen to know that node 375 is of this class (containing a cubic SrTiO3 structure).

In [221]:
strucdata = load_node(375)
print strucdata.get_attrs().keys()
print ''
print strucdata.get_attrs()['kinds']

[u'kinds', u'pbc3', u'pbc2', u'pbc1', u'sites', u'cell']

[{u'symbols': [u'Sr'], u'mass': 87.62, u'weights': [1.0], u'name': u'Sr'}, {u'symbols': [u'Ti'], u'mass': 47.867, u'weights': [1.0], u'name': u'Ti'}, {u'symbols': [u'O'], u'weights': [1.0], u'mass': 15.9994, u'name': u'O'}]


In [196]:
qb = QueryBuilder()
#We shall use this date as a cut-off date in a filter
date=datetime(2016,11,1)
StructureData = DataFactory('structure')

qb.append(
    StructureData,
    # We project out only the ID and 'kinds' columns of the DB
    project=["id","attributes.kinds"],
    # Filter by date created
    filters={"ctime": {">":date}})

print "Number of results returned: ", qb.count()

Number of results returned:  51


Now let's have a look at the first 10 results. We may be interested for example in what elements are present in the structure:

In [213]:
results = [i for i in qb.iterall()][:10]
for iD, item in results:
    elements = [basis['symbols'] for basis in item]
    elements = [val for sublist in elements for val in sublist]
    print iD, " ".join(elements)

22 Ba Ti O
112 Ba Ti O
118 Ba Ti O
142 Ba Ti O
129 Ba Ti O
133 Ba Ti O
137 Ba Ti O
169 Ba Ti O
147 Ba Ti O
153 Ba Ti O


## ParameterData

This subclass of Node contains parameter data, which can be either input parameters, or output parameters (results!).

Before we build a query, let's see what attributes this subclass contains. I happen to know that nodes 108 and 296 are input and output parameters.

In [110]:
#For example an input ParameterData:
inputparamnode = load_node(108)
inputparamnode.get_attrs()

{u'CONTROL': {u'calculation': u'scf',
  u'restart_mode': u'from_scratch',
  u'wf_collect': True},
 u'ELECTRONS': {u'conv_thr': 1e-06},
 u'SYSTEM': {u'ecutrho': 240.0, u'ecutwfc': 30.0}}

In [171]:
#For example an output ParameterData:
outputparamnode = load_node(296)
outputparamnode.get_attrs().keys()

[u'xml_warnings',
 u'monkhorst_pack_offset',
 u'non_colinear_calculation',
 u'symmetries',
 u'format_version',
 u'number_of_atoms',
 u'energy',
 u'rho_cutoff_units',
 u'energy_hartree',
 u'inversion_symmetry',
 u'energy_one_electron',
 u'magnetization_angle2',
 u'number_of_spin_components',
 u'magnetization_angle1',
 u'number_of_bands',
 u'fixed_occupations',
 u'wfc_cutoff_units',
 u'monkhorst_pack_grid',
 u'number_of_symmetries',
 u'spin_orbit_domag',
 u'pointgroup_international',
 u'fft_grid',
 u'fermi_energy',
 u'energy_one_center_paw',
 u'lsda',
 u'energy_accuracy_units',
 u'pp_check_flag',
 u'energy_ewald',
 u'no_time_rev_operations',
 u'wall_time',
 u'smearing_method',
 u'energy_ewald_units',
 u'non_local_df',
 u'number_of_species',
 u'wfc_cutoff',
 u'number_of_bravais_symmetries',
 u'parser_info',
 u'k_points_units',
 u'energy_one_electron_units',
 u'smooth_fft_grid',
 u'beta_real_space',
 u'constraint_mag',
 u'warnings',
 u'do_not_use_time_reversal',
 u'energy_threshold',
 u'en

Here let's try append a second thing to our query in order to only select those ParameterDatas for which a calculation was an input (i.e. output params).

In [215]:
qb = QueryBuilder()
date=datetime(2016,11,1)
ParameterData = DataFactory('parameter')

qb.append(
    ParameterData,
    # We need to tag this 'append' so we can refer to it in the second 'append'
    tag="outputparams",
    # Let's project out the ID and the total energy
    project=["id","attributes.energy"],
    filters={"ctime": {">":date}})

# Defining relationships: select those ParameterData that for which Computer is an input
qb.append(Calculation, input_of="outputparams")
print "Number of results returned: ", qb.count()

Number of results returned:  40


In [216]:
# Let's print out the first 10 results:
results = [i for i in qb.iterall()][:10]
for iD, energy in results:
    print "{0}   {1:7.4f}".format(iD, energy)

163   -3646.0172
198   -3646.0172
201   -3644.3705
195   -3644.1018
204   -3641.2883
296   -14619.8863
290   -14633.8570
212   -3646.0298
192   -3634.6463
293   -14632.7079


## KpointsData 

Self-explanatory... I know that node 377 has k-points data. Let's see what attributes it has:

In [118]:
kptsnode = load_node(377)
kptsnode.get_attrs()

{u'mesh': [4, 4, 4], u'offset': [0.0, 0.0, 0.0]}

In [217]:
qb = QueryBuilder()
date=datetime(2016,11,1)
KpointsData = DataFactory('array.kpoints')

qb.append(
    KpointsData,
    # We need to tag this 'append' so we can refer to it in the second 'append'
    tag="outputparams",
    # Let's project out the ID and the total energy
    project=["id","attributes.mesh"],
    filters={"ctime": {">":date}})

# Defining relationships: select those KpointsData that were inputs for a Computer
qb.append(Calculation, output_of="outputparams")
print "Number of results returned: ", qb.count()

Number of results returned:  42


In [218]:
# Let's print out the first 10 results:
results = [i for i in qb.iterall()][:10]

for iD, mesh in results:
    print "{0}   {1}".format(iD, mesh)

157   [4, 4, 4]
175   [4, 4, 4]
179   [4, 4, 4]
171   [4, 4, 4]
183   [4, 4, 4]
215   [4, 4, 4]
223   [4, 4, 4]
207   [4, 4, 4]
167   [4, 4, 4]
227   [4, 4, 4]


## UpfData 

This contains the pseudopotential information. For example, node 98 contains a pseudopotential for Ti.

In [159]:
upfnode = load_node(98)
print upfnode.get_attrs()
print upfnode.get_upf_family_names()

{u'filename': u'ti_pbesol_v1.4.uspp.F.UPF', u'md5': u'ebdc10ccce51265d8c030443845e563a', u'element': u'Ti'}
[u'SSSP']


In [163]:
qb = QueryBuilder()
date=datetime(2016,11,1)
UpfData = DataFactory('upf')

qb.append(
    UpfData,
    # We need to tag this 'append' so we can refer to it in the second 'append'
    tag="inputparams",
    # Let's project out the ID and the total energy
    project=["id","attributes.filename"],
    filters={"ctime": {">":date}})

# Defining relationships: select those KpointsData that were inputs for a Computer
qb.append(Calculation, output_of="inputparams")
print "Number of results returned: ", qb.count()

Number of results returned:  126


This is quite a lot of info, but for this purpose, let's just print the unique pseudopotential filenames:

In [168]:
fnames = [filename for iD, filename in qb.iterall()]
list(set(fnames))

[u'Sr.pbesol-spn-rrkjus_psl.1.0.0.UPF',
 u'Ba_ONCV_PBEsol-1.0.oncvpsp.upf',
 u'Pb.pbesol-dn-kjpaw_psl.0.2.2.UPF',
 u'o_pbesol_v1.2.uspp.F.UPF',
 u'ti_pbesol_v1.4.uspp.F.UPF']

## JobCalculation

Node 378 was a calculation job. 

In [170]:
calcnode = load_node(378)
calcnode.get_attrs().keys()

[u'retrieve_list',
 u'last_jobinfo',
 u'job_id',
 u'scheduler_state',
 u'parser',
 u'max_wallclock_seconds',
 u'jobresource_params',
 u'remote_workdir',
 u'state',
 u'linkname_retrieved',
 u'retrieve_singlefile_list',
 u'scheduler_lastchecktime']

In [175]:
qb = QueryBuilder()
date=datetime(2016,11,1)

qb.append(
    Calculation,
    # Let's project out the ID and the state
    project=["id","attributes.state", "attributes.job_id"],
    filters={"ctime": {">":date}})

print "Number of results returned: ", qb.count()

Number of results returned:  46


In [178]:
for iD, state, jobid in qb.iterall():
    print "{0}   {1}    {2}".format(iD, state, jobid)

159   FINISHED    4078535.sdb
176   FINISHED    4079024.sdb
180   FINISHED    4079026.sdb
172   FINISHED    4079023.sdb
184   FINISHED    4079027.sdb
216   FINISHED    4079127.sdb
224   FINISHED    4079125.sdb
208   FINISHED    4079043.sdb
168   FINISHED    4079021.sdb
228   FINISHED    4079126.sdb
220   FINISHED    4079131.sdb
252   FINISHED    4079137.sdb
244   FINISHED    4079135.sdb
260   FINISHED    4079139.sdb
248   FINISHED    4079136.sdb
256   FINISHED    4079138.sdb
268   FINISHED    4079141.sdb
232   FINISHED    4079130.sdb
272   FINISHED    4079142.sdb
236   FINISHED    4079145.sdb
264   FINISHED    4079140.sdb
240   FINISHED    4079133.sdb
324   FINISHED    4079213.sdb
334   FINISHED    4079214.sdb
390   FINISHED    4079434.sdb
360   SUBMISSIONFAILED    None
362   SUBMISSIONFAILED    None
370   RETRIEVALFAILED    4079430.sdb
398   FINISHED    4079436.sdb
344   FINISHED    4079215.sdb
374   RETRIEVALFAILED    4079429.sdb
382   FINISHED    4079432.sdb
358   SUBMISSIONFAILED  